# Candidate ranking model tutorial

`CandidateRankingModel` from RecTools is a fully funcitonal two-stage recommendation pipeline. 

On the first stage simple models generate candidates from their usual recommendations. On the second stage, a "reranker" (usually Gradient Boosting Decision Trees model) learns how to rank these candidates to predict user actual interactions.

Main features of our implementation:
- Ranks and scores from first-stage models can be added as features for the second-stage reranker.
- Explicit features for user-items candidate pairs can be added using `CandidateFeatureCollector`
- Custom negative samplers for creating second-stage train can be used.
- Custom splitters for creating second-stage train targets can be used.
- CatBoost models as second-stage reranking models are supported out of the box.

**You can treat `CandidateRankingModel` as any other RecTools model and easily pass it to cross-validation. All of the complicated logic for fitting first-stage and second-stage models and recommending through the whole pipeline will happen under the hood.**

**Table of Contents**

* Load data: kion
* Initialization of CandidateRankingModel
* What if we want to easily add user/item features to candidates?
    * From external source
* Using boosings from well-known libraries as a ranking model
    * CandidateRankingModel with gradient boosting from sklearn
        * Features of constructing model
    * CandidateRankingModel with gradient boosting from catboost
        * Features of constructing model
        * Using CatBoostClassifier
        * Using CatBoostRanker
    * CandidateRankingModel with gradient boosting from lightgbm
        * Features of constructing model
        * Using LGBMClassifier
        * Using LGBMRanker
            * An example of creating a custom class for reranker
* CrossValidate
    * Evaluating the metrics of candidate ranking models and candidate generator models

In [5]:
from rectools.models import PopularModel, ImplicitItemKNNWrapperModel
from implicit.nearest_neighbours import CosineRecommender
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset
from sklearn.linear_model import RidgeClassifier
from pathlib import Path
import pandas as pd
import numpy as np
from rectools import Columns
from lightgbm import LGBMClassifier, LGBMRanker
from catboost import CatBoostClassifier, CatBoostRanker
from sklearn.ensemble import GradientBoostingClassifier
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.model_selection import cross_validate
from rectools.models.ranking import (
    CandidateRankingModel,
    CandidateGenerator,
    Reranker,
    CatBoostReranker, 
    CandidateFeatureCollector,
    PerUserNegativeSampler
)
from rectools.models.base import ExternalIds
import typing as tp

## Load data: kion

In [4]:
%%time
!wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
!unzip -o data_original.zip
!rm data_original.zip

Archive:  data_original.zip
   creating: data_original/
  inflating: data_original/interactions.csv  
  inflating: __MACOSX/data_original/._interactions.csv  
  inflating: data_original/users.csv  
  inflating: __MACOSX/data_original/._users.csv  
  inflating: data_original/items.csv  
  inflating: __MACOSX/data_original/._items.csv  
CPU times: user 644 ms, sys: 183 ms, total: 827 ms
Wall time: 49.3 s


In [7]:
# Prepare dataset

DATA_PATH = Path("data_original")
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = (
    pd.read_csv(DATA_PATH / 'interactions.csv', parse_dates=["last_watch_dt"])
    .rename(columns={"last_watch_dt": Columns.Datetime})
)
interactions["weight"] = 1

In [8]:
dataset = Dataset.construct(interactions)

In [6]:
RANDOM_STATE = 32

## Initialization of `CandidateRankingModel`

In [10]:
# Prepare first stage models. They will be used to generate candidates for reranking
first_stage = [
    CandidateGenerator(PopularModel(), num_candidates=30, keep_ranks=True, keep_scores=True), 
    CandidateGenerator(
        ImplicitItemKNNWrapperModel(CosineRecommender()), 
        num_candidates=30, 
        keep_ranks=True, 
        keep_scores=True
    )
]

In [ ]:
# Prepare reranker. This model is used to rerank candidates from first stage models. 
# It is usually trained on classification or ranking task

reranker = CatBoostReranker()

In [9]:
# Prepare splitter for selecting reranker train. Only one fold is expected!
# This fold data will be used to define targets for training

splitter = TimeRangeSplitter("7D")

In [11]:
# Initialize CandidateRankingModel
# We can also pass negative sampler but here we are just using the default one

two_stage = CandidateRankingModel(first_stage, splitter, reranker)

## What data is reranker trained on? 

We can explicitly call `get_train_with_targets_for_reranker` method to look at the actual "train" for reranker.

Here' what happends under the hood during this call:
- Dataset interactions are split using provided splitter (usually on time basis) to history dataset and holdout interactions
- First stage models are fitted on history dataset
- First stage models generate recommendations -> These pairs become candidates for reranker
- All candidate pairs are assigned targets from holdout interactions. (`1` if interactions actually happend, `0` otherwise)
- Negative targets are sampled (here defult PerUserNegativeSampler is used which keeps a fixed number of negative samples per user)



In [12]:
candidates = two_stage.get_train_with_targets_for_reranker(dataset)

In [13]:
# This is train data for boosting model or any other reranker. id columns will be dropped before training
# Here we see ranks and scores from first-stage models as features for reranker
candidates.head(20)

,user_id,item_id,PopularModel_1_score,PopularModel_1_rank,ImplicitItemKNNWrapperModel_1_score,ImplicitItemKNNWrapperModel_1_rank,target
0,681331,12192,31907.0,14.0,0.120493,8.0,0
1,947281,7626,13131.0,29.0,0.477589,11.0,0
2,246422,9996,35718.0,10.0,0.194220,11.0,0
3,476975,7793,15221.0,23.0,NaN,NaN,0
4,417273,4471,NaN,NaN,0.148052,22.0,0
5,212338,4880,53191.0,6.0,0.885866,6.0,0
6,114667,9996,35718.0,10.0,0.124051,18.0,0
7,517345,12995,21577.0,11.0,0.725781,10.0,1
8,307295,10440,189923.0,1.0,0.089551,2.0,0
9,64657,13865,115095.0,1.0,1.876046,1.0,0


## What if we want to easily add user/item features to candidates?

You can add any user, item or user-item-pair features to candidates. They can be added from dataset or from external sources and they also can be time-dependent (e.g. item popularity).

To let the CandidateRankingModel join these features to train data for reranker, you need to create a custom feature collector. Inherit if from `CandidateFeatureCollector` which is used by default.

You can overwrite the following methods:
- `_get_user_features`
- `_get_item_features`
- `_get_user_item_features`

Each of the methods receives:
- `dataset` with all interactions that are available for model in this particular moment (no leak from the future). You can use it to collect user ot items stats on the current moment.
- `fold_info` with fold stats if you need to know that date that model considers as current date. You can join time-dependant features from external source that are valid on this particular date.

In the example below we will simply collect users age, sex and income features from external csv file:

In [17]:
# Write custome feature collecting funcs for users, items and user/item pairs
class CustomFeatureCollector(CandidateFeatureCollector):
    
    def __init__(self, cat_cols: tp.List[str])-> None:        
        self.cat_cols = cat_cols
    
    # your any helper functions for working with loaded data
    def _encode_cat_cols(self, df: pd.DataFrame) -> pd.DataFrame:    
        df_cat_cols = self.cat_cols
        df[df_cat_cols] = df[df_cat_cols].astype("category")

        for col in df_cat_cols:
            cat_col = df[col].astype("category").cat
            df[col] = cat_col.codes.astype("category")
        return df
    
    def _get_user_features(
        self, users: ExternalIds, dataset: Dataset, fold_info: tp.Optional[tp.Dict[str, tp.Any]]
    ) -> pd.DataFrame:
        columns = self.cat_cols.copy()
        columns.append(Columns.User)
        user_features = pd.read_csv(DATA_PATH / "users.csv")[columns]        
        
        users_without_features = pd.DataFrame(
            np.setdiff1d(dataset.user_id_map.external_ids, user_features[Columns.User].unique()),
            columns=[Columns.User]
        )        
        user_features = pd.concat([user_features, users_without_features], axis=0)
        user_features = self._encode_cat_cols(user_features)
        
        return user_features[user_features[Columns.User].isin(users)]

In [18]:
# Now we specify our custom feature collector for CandidateRankingModel

two_stage = CandidateRankingModel(
    first_stage,
    splitter,
    Reranker(RidgeClassifier()),
    feature_collector=CustomFeatureCollector(cat_cols = ["age", "income", "sex"])
)

In [19]:
candidates = two_stage.get_train_with_targets_for_reranker(dataset)

In [20]:
# Now our candidates also have features for users: age, sex and income
candidates.head(20)

,user_id,item_id,PopularModel_1_score,PopularModel_1_rank,ImplicitItemKNNWrapperModel_1_score,ImplicitItemKNNWrapperModel_1_rank,target,age,income,sex
0,168379,11640,NaN,NaN,0.144429,13.0,0,0,2,0
1,462121,3734,69687.0,6.0,NaN,NaN,1,0,2,0
2,826617,14809,NaN,NaN,0.147328,4.0,1,0,2,0
3,184867,2657,66415.0,7.0,NaN,NaN,0,2,3,1
4,716827,4436,16846.0,23.0,NaN,NaN,0,5,2,1
5,729424,10440,189923.0,1.0,NaN,NaN,0,0,2,0
6,1080167,11863,16231.0,18.0,NaN,NaN,0,-1,-1,-1
7,22315,13865,115095.0,3.0,0.403324,6.0,0,1,2,0
8,865689,7107,16279.0,27.0,NaN,NaN,0,1,2,0
9,276952,9728,119797.0,3.0,NaN,NaN,0,1,3,0


## Using boosings from well-known libraries as a ranking model

### CandidateRankingModel with gradient boosting from sklearn

**Features of constructing model:**
   - `GradientBoostingClassifier` works correctly with Reranker
   - `GradientBoostingClassifier` cannot work with missing values. When initializing CandidateGenerator, specify the parameter values `scores_fillna_value` and `ranks_fillna_value`.

In [21]:
# Prepare first stage models
first_stage_gbc = [
    CandidateGenerator(
        model=PopularModel(),
        num_candidates=30,
        keep_ranks=True,
        keep_scores=True,
        scores_fillna_value=1.01, # when working with the GradientBoostingClassifier, you need to fill in the empty scores (e.g. max score)
        ranks_fillna_value=31  # when working with the GradientBoostingClassifier, you need to fill in the empty ranks (e.g. min rank)
    ), 
    CandidateGenerator(
        model=ImplicitItemKNNWrapperModel(CosineRecommender()),
        num_candidates=30,
        keep_ranks=True,
        keep_scores=True,
        scores_fillna_value=1.01, # when working with the GradientBoostingClassifier, you need to fill in the empty scores (e.g. max score)
        ranks_fillna_value=31  # when working with the GradientBoostingClassifier, you need to fill in the empty ranks (e.g. min rank)
    )
]

In [22]:
two_stage_gbc = CandidateRankingModel(
    first_stage_gbc,
    splitter,
    Reranker(GradientBoostingClassifier(random_state=RANDOM_STATE)),
    sampler=PerUserNegativeSampler(n_negatives=3, random_state=RANDOM_STATE) # pass sampler to fix random_state
)

In [23]:
two_stage_gbc.fit(dataset)

In [24]:
reco_gbc = two_stage_gbc.recommend(
    users=dataset.user_id_map.external_ids, 
    dataset=dataset,
    k=10,
    filter_viewed=True
)

In [25]:
reco_gbc.head(5)

,user_id,item_id,score,rank
0,1097557,10440,0.613872,1
1,1097557,13865,0.506201,2
2,1097557,9728,0.472571,3
3,1097557,3734,0.349941,4
4,1097557,2657,0.287745,5


### CandidateRankingModel with gradient boosting from catboost

**Features of constructing model:**
- for `CatBoostClassifier` and `CatBoostRanker` it is necessary to process categorical features: fill in empty values (if there are categorical features in the training sample for Rerankers). You can do this with CustomFeatureCollector.

**Using CatBoostClassifier**
- `CatBoostClassifier` works correctly with CatBoostReranker

In [26]:
# Prepare first stage models
first_stage_catboost = [
    CandidateGenerator(
        model=PopularModel(),
        num_candidates=30,
        keep_ranks=True,
        keep_scores=True,
    ), 
    CandidateGenerator(
        model=ImplicitItemKNNWrapperModel(CosineRecommender()),
        num_candidates=30,
        keep_ranks=True,
        keep_scores=True,
    )
]

In [27]:
cat_cols = ["age", "income", "sex"]

# Categorical features are definitely transferred to the pool_kwargs
pool_kwargs = {
    "cat_features": cat_cols    
}

In [28]:
# To transfer CatBoostClassifier we use CatBoostReranker (for faster work with large amounts of data)
# You can also pass parameters in fit_kwargs and pool_kwargs in CatBoostReranker

two_stage_catboost_classifier = CandidateRankingModel(
    candidate_generators=first_stage_catboost,
    splitter=splitter,
    reranker=CatBoostReranker(CatBoostClassifier(verbose=False, random_state=RANDOM_STATE), pool_kwargs=pool_kwargs),
    sampler=PerUserNegativeSampler(n_negatives=3, random_state=RANDOM_STATE) # pass sampler to fix random_state
    feature_collector=CustomFeatureCollector(cat_cols)
)

In [29]:
two_stage_catboost_classifier.fit(dataset)

In [30]:
reco_catboost_classifier = two_stage_catboost_classifier.recommend(
    users=dataset.user_id_map.external_ids, 
    dataset=dataset,
    k=10,
    filter_viewed=True
)

In [31]:
reco_catboost_classifier.head(5)

,user_id,item_id,score,rank
0,1097557,10440,0.590609,1
1,1097557,7417,0.585314,2
2,1097557,9728,0.454810,3
3,1097557,13865,0.453770,4
4,1097557,3734,0.364262,5


**Using CatBoostRanker**
- `CatBoostRanker` works correctly with CatBoostReranker

In [32]:
# To transfer CatBoostRanker we use CatBoostReranker

two_stage_catboost_ranker = CandidateRankingModel(
    candidate_generators=first_stage_catboost,
    splitter=splitter,
    reranker=CatBoostReranker(CatBoostRanker(verbose=False, random_state=RANDOM_STATE), pool_kwargs=pool_kwargs),
    sampler=PerUserNegativeSampler(n_negatives=3, random_state=RANDOM_STATE) # pass sampler to fix random_state
    feature_collector=CustomFeatureCollector(cat_cols),                
)

In [33]:
two_stage_catboost_ranker.fit(dataset)

In [34]:
reco_catboost_ranker = two_stage_catboost_ranker.recommend(
    users=dataset.user_id_map.external_ids, 
    dataset=dataset,
    k=10,
    filter_viewed=True
)

In [35]:
reco_catboost_ranker.head(5)

,user_id,item_id,score,rank
0,1097557,10440,2.420927,1
1,1097557,13865,1.738958,2
2,1097557,9728,1.571645,3
3,1097557,3734,1.190009,4
4,1097557,142,1.030506,5


### CandidateRankingModel with gradient boosting from lightgbm
**Features of constructing model:**
- `LGBMClassifier` and `LGBMRanker` cannot work with missing values

**Using LGBMClassifier**
- `LGBMClassifier` works correctly with Reranker

In [36]:
# Prepare first stage models
first_stage_lgbm = [
    CandidateGenerator(
        model=PopularModel(),
        num_candidates=30,
        keep_ranks=True,
        keep_scores=True,
        scores_fillna_value=1.01, # when working with the LGBMClassifier, you need to fill in the empty scores (e.g. max score)
        ranks_fillna_value=31  # when working with the LGBMClassifier, you need to fill in the empty ranks (e.g. min rank)
    ), 
    CandidateGenerator(
        model=ImplicitItemKNNWrapperModel(CosineRecommender()),
        num_candidates=30,
        keep_ranks=True,
        keep_scores=True,
        scores_fillna_value=1,  # when working with the LGBMClassifier, you need to fill in the empty scores
        ranks_fillna_value=31   # when working with the LGBMClassifier, you need to fill in the empty ranks
    )
]

In [37]:
cat_cols = ["age", "income", "sex"]

# example parameters for running model training 
# more valid parameters here https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier.fit
fit_params = {
    "categorical_feature": cat_cols,
}

In [38]:
two_stage_lgbm_classifier = CandidateRankingModel(
    candidate_generators=first_stage_lgbm,
    splitter=splitter,
    reranker=Reranker(LGBMClassifier(random_state=RANDOM_STATE), fit_params),
    sampler=PerUserNegativeSampler(n_negatives=3, random_state=RANDOM_STATE) # pass sampler to fix random_state
    feature_collector=CustomFeatureCollector(cat_cols)
)

In [39]:
two_stage_lgbm_classifier.fit(dataset)

[LightGBM] [Info] Number of positive: 78233, number of negative: 330228
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 395
[LightGBM] [Info] Number of data points in the train set: 408461, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191531 -> initscore=-1.440092
[LightGBM] [Info] Start training from score -1.440092


In [40]:
reco_lgbm_classifier = two_stage_lgbm_classifier.recommend(
    users=dataset.user_id_map.external_ids, 
    dataset=dataset,
    k=10,
    filter_viewed=True
)

In [41]:
reco_lgbm_classifier.head(5)

,user_id,item_id,score,rank
0,1097557,10440,0.610178,1
1,1097557,13865,0.510029,2
2,1097557,9728,0.479905,3
3,1097557,3734,0.347386,4
4,1097557,2657,0.290810,5


**Using LGBMRanker**
- `LGBMRanker` does not work correctly with Reranker!

When using LGBMRanker, you need to correctly compose groups. To do this, you can create a class inheriting from Reranker and override method `prepare_fit_kwargs` in it.

Documentation on how to form groups for LGBMRanker (read about `group`):
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRanker.html#lightgbm.LGBMRanker.fit

**An example of creating a custom class for reranker**

In [42]:
class LGBMReranker(Reranker):
    def __init__(
        self,
        model: LGBMRanker,
        fit_kwargs: tp.Optional[tp.Dict[str, tp.Any]] = None,
    ):
        super().__init__(model)
        self.fit_kwargs = fit_kwargs
        
    def _get_group(self, df: pd.DataFrame) -> np.ndarray:
        return df.groupby(by=["user_id"])["item_id"].count().values

    def prepare_fit_kwargs(self, candidates_with_target: pd.DataFrame) -> tp.Dict[str, tp.Any]:
        candidates_with_target = candidates_with_target.sort_values(by=[Columns.User])
        groups = self._get_group(candidates_with_target)
        candidates_with_target = candidates_with_target.drop(columns=Columns.UserItem)

        
        fit_kwargs = {
            "X": candidates_with_target.drop(columns=Columns.Target),
            "y": candidates_with_target[Columns.Target],
            "group": groups,
        }

        if self.fit_kwargs is not None:
            fit_kwargs.update(self.fit_kwargs)

        return fit_kwargs

In [43]:
cat_cols = ["age", "income", "sex"]

# example parameters for running model training 
# more valid parameters here
# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRanker.html#lightgbm.LGBMRanker.fit
fit_params = {
    "categorical_feature": cat_cols,
}

In [44]:
# Now we specify our custom feature collector for CandidateRankingModel

two_stage_lgbm_ranker = CandidateRankingModel(
    candidate_generators=first_stage_lgbm,
    splitter=splitter,
    reranker=LGBMReranker(LGBMRanker(random_state=RANDOM_STATE), fit_kwargs=fit_params),
    sampler=PerUserNegativeSampler(n_negatives=3, random_state=RANDOM_STATE) # pass sampler to fix random_state
    feature_collector=CustomFeatureCollector(cat_cols)
    )

In [45]:
two_stage_lgbm_ranker.fit(dataset)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003223 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 396
[LightGBM] [Info] Number of data points in the train set: 408461, number of used features: 7


In [46]:
reco_lgbm_ranker = two_stage_lgbm_ranker.recommend(
    users=dataset.user_id_map.external_ids, 
    dataset=dataset,
    k=10,
    filter_viewed=True
)

In [47]:
reco_lgbm_ranker.head(5)

,user_id,item_id,score,rank
0,1097557,10440,2.095641,1
1,1097557,13865,1.503235,2
2,1097557,9728,1.420993,3
3,1097557,3734,0.806803,4
4,1097557,142,0.725385,5


## CrossValidate
### Evaluating the metrics of candidate ranking models and candidate generator models.

In [48]:
# Take few models to compare
models = {
    "popular": PopularModel(),
    "cosine_knn": ImplicitItemKNNWrapperModel(CosineRecommender()),
    "two_stage_gbc": two_stage_gbc,
    "two_stage_catboost_classifier": two_stage_catboost_classifier,
    "two_stage_catboost_ranker": two_stage_catboost_ranker,
    "two_stage_lgbm_classifier": two_stage_lgbm_classifier,
    "two_stage_lgbm_ranker": two_stage_lgbm_ranker
}

# We will calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@1": Precision(k=1),
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty@10": MeanInvUserFreq(k=10),
    "serendipity@10": Serendipity(k=10),
}

K_RECS = 10

In [49]:
%%time

cv_results = cross_validate(
    dataset=dataset,
    splitter=splitter,
    models=models,
    metrics=metrics,
    k=K_RECS,
    filter_viewed=True,
)

[LightGBM] [Info] Number of positive: 73891, number of negative: 310533
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002992 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 394
[LightGBM] [Info] Number of data points in the train set: 384424, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.192212 -> initscore=-1.435699
[LightGBM] [Info] Start training from score -1.435699
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003532 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 395
[LightGBM] [Info] Number of data points in the train set: 384424, number of used features: 7
CPU times: user 23min, sys: 51.8 s, total: 23min 52s
Wall time: 8min 49s


In [50]:
pivot_results = (
    pd.DataFrame(cv_results["metrics"])
    .drop(columns="i_split")
    .groupby(["model"], sort=False)
    .agg(["mean"])
)
pivot_results

,prec@1,prec@10,recall@10,novelty@10,serendipity@10
,mean,mean,mean,mean,mean
model,,,,,
popular,0.070806,0.032655,0.166089,3.715659,0.000002
cosine_knn,0.079372,0.036757,0.176609,5.758660,0.000189
two_stage_gbc,0.085623,0.039609,0.194438,4.831911,0.000155
two_stage_catboost_classifier,0.084460,0.038667,0.189490,4.897715,0.000154
two_stage_catboost_ranker,0.088711,0.039578,0.193905,4.863340,0.000155
two_stage_lgbm_classifier,0.086795,0.039282,0.192634,4.843057,0.000154
two_stage_lgbm_ranker,0.087085,0.039757,0.195510,4.754899,0.000144
